In [1]:
import pandas as pd
import numpy as np
from data_generator.batch_generator import BatchGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint#, TensorBoard
import pickle
from models import AlexNet, LeNetRegressor
from keras import backend as K
K.set_image_data_format('channels_last')

/home/nicoli/github/alexnet/data_generator/batch_generator.py:42: UserWarning: 'BeautifulSoup' module is missing. The XML-parser will be unavailable.
  warnings.warn("'BeautifulSoup' module is missing. The XML-parser will be unavailable.")
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
test_dataset = BatchGenerator(box_output_format=['class_id'])

In [3]:
test_dataset.parse_csv(labels_filename='/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_age_regression_test_split_47950-70-10-20.csv', 
                        images_dir='/home/nicoli/github/ssd_keras/dataset/imdb-hand-crop/',
                        input_format=['image_name', 'class_id'])

In [4]:
img_height, img_width, img_depth = (224,224,3)

#epochs = 90

batch_size=8
shuffle = True
ssd_train = False

validation_batch_size = 32


# In[15]:

test_generator = test_dataset.generate(batch_size=batch_size,
                                         shuffle=shuffle,
                                         ssd_train=ssd_train,
                                         returns={'processed_labels'},
                                         resize=(img_height, img_width))

In [5]:
print("Number of images in the dataset:", test_dataset.get_n_samples())

Number of images in the dataset: 9590


In [6]:
steps = test_dataset.get_n_samples()/batch_size

## Alexnet

In [7]:
alexnet = AlexNet(1, img_width, img_height, img_depth, weights_path='callbacks/alexnet/age/weights.15-62040670.75.hdf5')


# In[7]:


alexnet.model.summary()
#alexnet.model.load_weights('callbacks/alexnet/age/weights.24-1658.03.hdf5')


/home/nicoli/github/alexnet/models.py:138: UserWarning: Considering a regression task with output function being 'relu'
  warnings.warn('Considering a regression task with output function being \'relu\'')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 56, 56, 96)        34944     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 56, 56, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 27, 27, 256)       614656    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 27, 27, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 27, 27, 256)       1024      
__________

In [8]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0000000001, amsgrad=True)

In [9]:
alexnet.model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

In [10]:
prev = next(test_generator)

In [16]:
predict = alexnet.model.predict(prev[0])

In [14]:
predict

array([[-12679.325 ],
       [ -4133.9097],
       [-26492.44  ],
       [ -4333.5107],
       [ -2881.6836],
       [-43764.45  ],
       [ -3635.1375],
       [ -2423.0227]], dtype=float32)

In [15]:
prev[1]

array([[36],
       [50],
       [45],
       [34],
       [64],
       [41],
       [41],
       [56]])

In [ ]:
stats3 = alexnet.model.evaluate_generator(test_generator,steps=steps)

In [ ]:
stats3

In [16]:
from math import sqrt
sqrt(stats3[0])

16.850956246369883

## Lenet

In [69]:
lenet = LeNetRegressor(1, img_width, img_height, img_depth, weights_path='callbacks/alexnet/age/weights.13-267.87.hdf5')


# In[7]:


lenet.model.summary()
#alexnet.model.load_weights('callbacks/alexnet/age/weights.24-1658.03.hdf5')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 220, 220, 6)       456       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 110, 110, 6)       0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 106, 106, 16)      2416      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 53, 53, 16)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 44944)             0         
_________________________________________________________________
dense_22 (Dense)             (None, 120)               5393400   
_________________________________________________________________
dense_23 (Dense)             (None, 84)                10164     
__________

/home/nicoli/github/alexnet/models.py:62: UserWarning: Considering a regression task with output function being 'relu'
  warnings.warn('Considering a regression task with output function being \'relu\'')


In [70]:
lenet.model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

In [71]:
prev = next(test_generator)

In [72]:
predict = lenet.model.predict(prev[0])

In [73]:
predict

array([[-0.8592841 ],
       [-0.35129142],
       [ 0.21100695],
       [-0.7630777 ],
       [-0.21265048],
       [-0.978853  ],
       [ 0.29091817],
       [-0.37627503]], dtype=float32)

In [74]:
prev[1]

array([[59],
       [43],
       [32],
       [47],
       [46],
       [26],
       [30],
       [27]])

In [75]:
stats4 = lenet.model.evaluate_generator(test_generator,steps=steps)

In [76]:
from math import sqrt
sqrt(stats4[0])

41.55802168214639

In [78]:
stats4

[1727.0691661337494, 38.98530484021518]